# Demonstration on 6x6 images

Important: The current code I'm using has moved into [toysamples1.py](toysamples1.py) for now, because displaying the output images here was eating up too much web browser memory.  It might move back at some point, since I could simply run that code from here, now that the images are now saved to files; and then just display images I want there.  But then, that would seem to negate a lot of the advantages of jupyter.  Anyway, we shall see.

In [ ]:
import numpy as np

print(np.random.choice(2, size=(4,)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math


num_classes = 4
image_size = 6
N = 100
num_samples_to_print = 7
sigma_X = 0.01

class_descriptions = [
    {
        'sprite': [
            [0,1,0],
            [1,1,1],
            [0,1,0]
        ],
        'sprite_x': 0,
        'sprite_y': 0
    },
    {
        'sprite': [
            [1,1,1],
            [1,0,1],
            [1,1,1]
        ],
        'sprite_x': 3,
        'sprite_y': 0
    },
    {
        'sprite': [
            [1,0,0],
            [1,1,0],
            [1,1,1]
        ],
        'sprite_x': 0,
        'sprite_y': 3
    },
    {
        'sprite': [
            [1,1,1],
            [0,1,0],
            [0,1,0]
        ],
        'sprite_x': 3,
        'sprite_y': 3
    }
]

def class_descriptions_to_class_pics():
    class_pics = []
    for i, desc in enumerate(class_descriptions):
        if i >= num_classes:
            break
        pic = np.zeros((image_size, image_size), dtype=np.float32)
        for dx in range(3):
            for dy in range(3):
                if desc['sprite'][dy][dx] == 1:
                    pic[desc['sprite_y'] + dy, desc['sprite_x'] + dx] = 1.0
        class_pics.append(pic)
    return class_pics

class_pics = class_descriptions_to_class_pics()
# print(class_pics)

def print_images(titles, images):
    plt.figure(1)
    num_figures = len(images)
    for i, image in enumerate(images):
        image_size = image.shape[0]
        image_min = np.min(image)
        image_max = np.max(image)
        image_min = -1
        image_max = 2
        image_range = image_max - image_min
        image = np.maximum(image_min, image)
        image = np.minimum(image_max, image)
        image = (image - image_min) / image_range
        image_rgb = np.zeros((image_size, image_size, 3), dtype=np.float32)
        image_rgb[:,:,0] = image
        image_rgb[:,:,1] = image
        image_rgb[:,:,2] = image
        plt.subplot(1, num_figures, i + 1)
        plt.imshow(image_rgb, interpolation='nearest')
        plt.axis('off')
        if titles[i] is not None:
            plt.title(titles[i])
    plt.show()

def draw_samples(N, class_pics):
    samples = []
    ground_truth_Z = np.zeros((N, num_classes), dtype=np.int8)
    samples_to_print = set(np.random.choice(N, (num_samples_to_print,), replace=False))
    for n in range(N):
        image = np.zeros((image_size, image_size), dtype=np.float32)
        features = np.random.choice(2, size=(num_classes,))
        ground_truth_Z[n] = features
#         print(features)
        for k, v in enumerate(features):
            if v == 1:
                image += class_pics[k]
        image_orig = np.copy(image)
#         print_image(image)

        noise = np.random.randn(image_size, image_size).astype(np.float32) * sigma_X
#         print_image(noise)

        image += noise
        if n in samples_to_print:
#             print(n)
            print_images(['orig', 'noise', 'noised image'], [image_orig, noise, image])

        samples.append(image)
    return samples, ground_truth_Z

samples, ground_truth_Z = draw_samples(N, class_pics)
# print('ground_truth_Z\n', ground_truth_Z)


In [ ]:
Z_columns = []
column = np.random.choice(2, (N,))
Z_columns.append(column)
K_plus = len(Z_columns)
# print(Z_columns)
M = []
M.append(np.sum(Z_columns[0]))
# print('M', M)

sigma_A = 0.5
sigma_X = 1.7
alpha = 1.0

for it in range(10):
    for n in range(N):
        k = 0
        while k < len(Z_columns):
            column = Z_columns[k]
            m = M[k]
            if m == 1 and column[n] == 1:
                # means that m_{-i} is zero
                # so, delete the column
                del Z_columns[k]
                del M[k]
                k -= 1
            else:
                # M_{-i} > 0
                # what do we do now???
                pass

            k += 1

# note that this code block wont do anything useful :-)  I abandoned it.
# A new one, based on this, is much lower down this page.

## Interlude: accelerated gibbs sampling

At this point, I looked at the tutorial's equation 22.  But there's a proportional-to sign, $\propto$.  How to handle this?  Since I'm not trying to do this research from scratch, just reproduce/understand the existing research, I reached out to Mr Google, to look for more explanations.  I found the following video, from Finale Doshi-Velez, which I'm going to interlude out to now.  The interlude is at: [accelerated_gibbs_samplings.ipynb](accelerated_gibbs_sampling.ipynb).


## Demonstration on 6x6 images, revisited

### Challenge: sampling from non-normalized distribution?

From reaching out to Doshi-Velez's presentation, it looks like we can sample a distribution that is only provided in proportionality, as long as it has a shape, which is well-defined, that we know how to sample from, ie typically a Gaussian.  There are probably other ways of handling other distributions, but a Gaussian should be reasonably straightforward to sample from, if we can get the distribution in that form.  Let's reach right back to the first section, and see what distribution(s) we need to sample from.

Equation 22 from the Griffiths and Ghahramani tutorial states:

$$
P(z_{ik} \mid \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
\propto
p(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
\,
P(z_{ik} \mid \mathbf{z}_{-i,k})
$$

The first term of this equation, ie the likelihood of $\mathbf{X}$, given the latent variables, and the hyper-parameters, is a Gaussian.  For the finite model, $P(z_{ik} \mid \mathbf{Z}_{-i,k})$ is given by equation 17 in the tutorial:

$$
P(z_{ik} = 1 \mid \mathbf{z}_{-i,k})
= \frac{m_{-i,k} + \frac{\alpha}{K}}
  {N + \frac{\alpha}{K}}
$$

This seems not to be a Gaussian.  How to sample from the product of a Gaussian and this term?

### Interlude: is the product of two normalized distributions also normalized?

Brainstorming a bit, we could sample from the Gaussian, which we could normalize first, and then multiply by $p(z_{ik} = 1 \mid \mathbf{z}_{-i,k})$.  Is it fair to say that the product of two normalized probability functions will be normalized?  Probably not, eg we could have the following two distributions:

$$
f(x) = 1
\mathrm{\,when\,} x \ge 0 \mathrm{\,and\,} x \le 1 \\ 
= 0 \mathrm{\, otherwise}
$$

(which integrates to 1), and:

$$
g(x) = 1
\mathrm{\,when\,} x \ge 2 \mathrm{\,and\,} x \le 3 \\ 
= 0 \mathrm{\, otherwise}
$$

... which integrates to 1 too.  But their product integrates to 0.

### Integrate the un-normalized distribution over $z_{ik}$?

Actually, the equation for the probaiblty of $z_{ik} = 1$ is not actually a probability distribution: it's the value of this probaiblity for one specific value of $z_{ik}$, ie $1$.

Let's try integrating over $c \cdot p(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A) \cdot P(z_{ik} \mid \mathbf{z}_{-i,k})$ $z_{ik}$, using a probability distribution of $z_{ik}$, rather than just one specific value, and where $c$ is a constant of normalization, that will make the integrant integrate to $1$.

$$
\int
c
\cdot
P(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
\cdot
P(z_{ik})
\,
dz_{ik}
$$

And since $z_{ik}$ is discrete, ie $z_{ik} \in \{0, 1\}$, then we can rewrite the integral as a sum:

$$
=
c
\sum_{z_{ik}=0}^1
\left(
    P(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
    \cdot
    P(z_{ik})
\right)
$$
&nbsp;

$$
=
c
\sum_{z_{ik}=0}^1
\left(
    \mathcal{N}(\mathbf{X}; \mu_{\mathbf{Z}, \sigma_A, \sigma_X}, \Sigma_{\mathbf{Z}, \sigma_A, \sigma_X})
    \cdot
    P(z_{ik})
\right)
$$



So, it seems like maybe we can simply calculate the value of the gaussian, for $z_{ik} \in \{0, 1\}$, and multiply by $P(z_{ik} \mid \mathbf{z}_{-i,k})$, each time; and then normalize the sum of these two products?  Just to imagine this a bit, let's say we have:

In [ ]:
import numpy as np

p_X_given_Z = [0.03, 0.02]  # pretend Gaussian samples, not normalized
p_zik_given_Z_minus = [0.8, 0.2]  # normalized, sum to 1.0

#Then
p_zik_given_X_Z = [0] * 2
for zik in [0, 1]:
    p_zik_given_X_Z[zik] = p_X_given_Z[zik] * p_zik_given_Z_minus[zik]

print(p_zik_given_X_Z)

# normalize
p_zik_given_X_Z /= np.sum(p_zik_given_X_Z)
print('normalized p_zik_given_X_Z', p_zik_given_X_Z)


So, the normalized values, with this toy data, are influenced by both the likelihood, and by the prior.

Let's run with this.

## Solving for expected A

We have:

$$
\mathbb{E}[\mathbf{A}] = (\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I})^{-1}\mathbf{Z}^T\mathbf{X}
$$

It'd be good to avoid that inverse.  Can we avoid it using a solver?  The numpy solver, [numpy.linalg.solve](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve) solves equations in the form:

$$
\mathbf{A} \mathbf{X} = \mathbf{B}
$$

So, let's put the equation in $\mathbb{E}[\mathbf{A}]$ above into this form:

$$
(\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I})\mathbb{E}[\mathbf{A}] = \mathbf{Z}^T\mathbf{X}
$$

So, using $\mathrm{\backslash}$ to denote "solve", we have:

$$
\mathbb{E}[\mathbf{A}] = \mathbf{Z}^T\mathbf{X} \mathrm{\,\backslash\,} (\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I})
$$


In [ ]:
import random

Z_columns = []
column = np.random.choice(2, (N,))
Z_columns.append(column)
K_plus = len(Z_columns)
# print(Z_columns)
M = []
M.append(np.sum(Z_columns[0]))
# print('M', M)

sigma_A = 1.0
sigma_X = 0.5
alpha = 1.0


def columns_to_array(columns):
    if len(columns) == 0:
        return None
    rows = columns[0].shape[0]
    cols = len(columns)
    array = np.zeros((rows, cols), dtype=np.float32)
    for col, column in enumerate(columns):
        array[:, col] = column
    return array


def calc_log_p_X_given_Z(Z_columns, X, sigma_X, sigma_A):
    Z = columns_to_array(Z_columns)
#     print('Z', Z)
    ZTZI = Z.T.dot(Z) + (sigma_X * sigma_X / sigma_A / sigma_A) * np.identity(Z.shape[1])
#     print('ZTZI', ZTZI)
    ZTZIInv = np.linalg.inv(ZTZI)
#     print('ZTZIInv', ZTZIInv)
    IZZZIZ = np.identity(Z.shape[0]) - Z.dot(ZTZIInv).dot(Z.T)
#     print('IZZZIZ\n', IZZZIZ)
    XT___X = X.T.dot(IZZZIZ).dot(X)
#     print('XT___X\n', XT___X)
    trace_term = np.trace(XT___X)
#     print('trace_term', trace_term)
    exponent = - 1 / (sigma_X * sigma_X * 2) * trace_term
#     print('exponent', exponent)
    return exponent
#     gaussian_unnorm = np.exp(exponent)
#     print('gaussian prob [%s]' % gaussian_unnorm)
#     return gaussian_unnorm


def calc_A(Z_columns, sigma_X, sigma_A):
    Z = columns_to_array(Z_columns)
    I = sigma_X * sigma_X / (sigma_A * sigma_A) * np.identity(Z.shape[1])
    ZTZI = Z.T.dot(Z) + I
#     print('ZTZI', ZTZI)
    ZTZIInv = np.linalg.inv(ZTZI)
#     print('ZTZIInv', ZTZIInv)
    E_A = ZTZIInv.dot(Z.T).dot(X)
#     print('E_A\n', E_A)
#     print('E_A.shape', E_A.shape)
    image_titles = []
    images = []
    for k in range(E_A.shape[0]):
        image_flat = E_A[k]
        image = image_flat.reshape(image_size, image_size)
        image_titles.append(None)
        images.append(image)
#         print_images(['A k=%s' % k], [image])
#     asdf
    print_images(image_titles, images)
    return E_A


def samples_to_X(samples):
    N = len(samples)
    X_features = samples[0].shape[0] * samples[0].shape[1]
    X = np.zeros((N, X_features), dtype=np.float32)
    for n, sample in enumerate(samples):
        X[n] = sample.reshape(X_features)
    return X


X = samples_to_X(samples)
print('X.shape', X.shape)
# print('X\n', X)
# print('ground truth Z\n', ground_truth_Z)
np.set_printoptions(suppress=False, precision=3)
num_its = 100
for it in range(num_its):
    num_added = 0
    num_removed = 0
    for n in range(N):
        k = 0
        while k < len(Z_columns):
            old_zik = Z_columns[k][n]
            if old_zik == 1:
                m_minusi_k = M[k] - 1
            else:
                m_minusi_k = M[k]
            if m_minusi_k > 0:
                # get the probabilty of z_ik given Z_minus_ik, for
                # zik = 0 and zik = 1
                p_zik_given_Zminus = np.zeros((2,), dtype=np.float32)
                p_zik_given_Zminus[1] = m_minusi_k / N
                p_zik_given_Zminus[0] = 1.0 - p_zik_given_Zminus[1]
                
                # and we need also to get the probability from the gaussian, again
                # for zik=0 and zik=1
                # for now, lets just stupidly calculate it, not do rank-1s or anything

                # calculate as log first, then normalize this first, then
                # exp it, to avoid crazily tiny values etc
                log_p_X_given_Z = np.zeros((2,), dtype=np.float32)
                for zik in [0, 1]:
                    Z_columns[k][n] = zik
                    # add epsilon to it, to avoid nans
                    log_p_X_given_Z[zik] = calc_log_p_X_given_Z(Z_columns, X, sigma_X, sigma_A)
#                 print('log_p_X_given_Z', log_p_X_given_Z)
                log_p_X_given_Z -= np.min(log_p_X_given_Z)
#                 print('log_p_X_given_Z norm', log_p_X_given_Z)
                p_X_given_Z = np.exp(log_p_X_given_Z)
#                 print('p_X_given_Z', p_X_given_Z)
                
#                 print('p_zik_given_Zminus', p_zik_given_Zminus)
#                 print('p_X_given_Z\n', p_X_given_Z)
                p_zik_given_X_Z_unnorm = np.multiply(
                    p_zik_given_Zminus, p_X_given_Z)
                p_zik_given_X_Z = p_zik_given_X_Z_unnorm / np.sum(p_zik_given_X_Z_unnorm)
#                 print('p_zik_given_X_Z', p_zik_given_X_Z)

                prob_zik_one = p_zik_given_X_Z[1]

                p = random.uniform(0, 1)
                new_zik = 1 if p <= prob_zik_one else 0
                Z_columns[k][n] = new_zik
                M[k] += new_zik - old_zik
            else:
                del M[k]
                del Z_columns[k]
                num_removed += 1
                k -= 1

            k += 1
        # add new features
        num_new_features = np.random.poisson(alpha / N)
        for j in range(num_new_features):
            M.append(1)
            new_col = np.zeros((N,), dtype=np.float32)
            new_col[n] = 1
            Z_columns.append(new_col)
            num_added += 1

    if (it + 1) % (num_its // 10) == 0:
        print('it %s' % (it + 1))
#         print(columns_to_array(Z_columns))
        expected_A = calc_A(Z_columns, sigma_X, sigma_A)
#         print('expected_A\n', expected_A)
#     print('Z_columns', Z_columns)